In [1]:
import torch
from torch import nn
import numpy as np
import torchvision.models as models
import torch.nn.functional as F

In [7]:
def bilinear_kernel(in_channels, out_channels, kernel_size):
    """
    The bilinear kernel estimates pixel values at new positions by taking weighted averages of nearby pixels. 
    The weights decrease linearly with distance from the interpolation point.
    """
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = (torch.arange(kernel_size).reshape(-1, 1),
          torch.arange(kernel_size).reshape(1, -1))
    filt = (1 - torch.abs(og[0] - center) / factor) * \
           (1 - torch.abs(og[1] - center) / factor)
    weight = torch.zeros((in_channels, out_channels,
                          kernel_size, kernel_size))
    weight[range(in_channels), range(out_channels), :, :] = filt # every channel initialize with the same one
    return weight

# conv_trans = nn.ConvTranspose2d(3, 3, kernel_size=4, padding=1, stride=2,
#                                 bias=False)
# conv_trans.weight.data.copy_(bilinear_kernel(3, 3, 4));
print(bilinear_kernel(1, 2, 4))
print(16 * 0.5625, 16 * 0.1875, 16 * 0.0625)

tensor([[[[0.0625, 0.1875, 0.1875, 0.0625],
          [0.1875, 0.5625, 0.5625, 0.1875],
          [0.1875, 0.5625, 0.5625, 0.1875],
          [0.0625, 0.1875, 0.1875, 0.0625]],

         [[0.0625, 0.1875, 0.1875, 0.0625],
          [0.1875, 0.5625, 0.5625, 0.1875],
          [0.1875, 0.5625, 0.5625, 0.1875],
          [0.0625, 0.1875, 0.1875, 0.0625]]]])
9.0 3.0 1.0


In [13]:
"""
Anchor sign to ground truth
"""
jaccard = torch.randn((6, 3))
print(jaccard)
anchors_bbox_map = torch.full((6,), -1, dtype=torch.long)
print("box_map:\n", anchors_bbox_map)
max_ious, indices = torch.max(jaccard, dim=1)
print("max_ious:\n", max_ious)
print("indices:\n", indices)
anc_i = torch.nonzero(max_ious >= 0.5).reshape(-1) # anchors with large iou
box_j = indices[max_ious >= 0.5]                   # [anc_i, box_j] is the anchors with large iou
print("anc_i:\n", anc_i)
print("box_j:\n", box_j)
anchors_bbox_map[anc_i] = box_j
print("box_map:\n", anchors_bbox_map)
col_discard = torch.full((6,), -1)
row_discard = torch.full((3,), -1)
for _ in range(3):
    max_idx = torch.argmax(jaccard)  # Find the largest IoU
    print(max_idx)
    box_idx = (max_idx % 3).long()
    anc_idx = (max_idx / 3).long()
    anchors_bbox_map[anc_idx] = box_idx
    jaccard[:, box_idx] = col_discard
    jaccard[anc_idx, :] = row_discard
print("col_discard:\n", col_discard)
print("row_discard:\n", row_discard)
print("box_map:\n", anchors_bbox_map)

tensor([[ 0.9518,  1.1292, -0.6121],
        [ 0.9860,  0.0299,  1.0424],
        [-0.2458, -0.2304,  0.1779],
        [-2.2789, -0.4275, -0.1470],
        [-0.9690, -0.3444,  1.6849],
        [ 0.7480, -2.2489, -0.6851]])
box_map:
 tensor([-1, -1, -1, -1, -1, -1])
max_ious:
 tensor([ 1.1292,  1.0424,  0.1779, -0.1470,  1.6849,  0.7480])
indices:
 tensor([1, 2, 2, 2, 2, 0])
anc_i:
 tensor([0, 1, 4, 5])
box_j:
 tensor([1, 2, 2, 0])
box_map:
 tensor([ 1,  2, -1, -1,  2,  0])
tensor(14)
tensor(1)
tensor(3)
col_discard:
 tensor([-1, -1, -1, -1, -1, -1])
row_discard:
 tensor([-1, -1, -1])
box_map:
 tensor([ 1,  0, -1, -1,  2,  0])


In [1]:
"""
Random shuffle the indices
"""
import random
indices = list(range(10))
print(indices)
random.shuffle(indices)
print(indices)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[6, 2, 4, 1, 5, 7, 0, 3, 8, 9]


In [24]:
"""
Wrapper will auto call the wrapper func
"""
class Animal:
    def __init__(self):
        self.name = "Animal"

def add_to_class(Class):
    """Register functions as methods in created class.

    Defined in :numref:`sec_oo-design`"""
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)
    return wrapper

@add_to_class(Animal)
def print_name(self):
    print(self.name)

def print_name_twice(self):
    print(self.name, self.name)

setattr(Animal, print_name_twice.__name__, print_name_twice)

animal = Animal()
animal.print_name()

animal.print_name_twice()

Animal
Animal Animal


In [21]:
"""
Wrapper will auto call the wrapper func
"""
def my_wrapper(obj):
    print("This is my wrapper!")
    return obj

@my_wrapper
def print_msg():
    print("This is print")

print_msg()

This is my wrapper!
This is print


In [28]:
import inspect
class HyperParameters:
    """The base class of hyperparameters."""
    def save_hyperparameters(self, ignore=[]):
        """Defined in :numref:`sec_oo-design`"""
        raise NotImplemented

    def save_hyperparameters(self, ignore=[]):
        """Save function arguments into class attributes.
    
        Defined in :numref:`sec_utils`"""
        frame = inspect.currentframe().f_back
        _, _, _, local_vars = inspect.getargvalues(frame)
        print(frame)
        print(inspect.getargvalues(frame))
        self.hparams = {k:v for k, v in local_vars.items()
                        if k not in set(ignore+['self']) and not k.startswith('_')}
        for k, v in self.hparams.items():
            setattr(self, k, v)

class ProgressBoard(HyperParameters):
    """The board that plots data points in animation.

    Defined in :numref:`sec_oo-design`"""
    def __init__(self, xlabel=None, ylabel=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 ls=['-', '--', '-.', ':'], colors=['C0', 'C1', 'C2', 'C3'],
                 fig=None, axes=None, figsize=(3.5, 2.5), display=True):
        self.save_hyperparameters()

pb = ProgressBoard()
print(pb.hparams)

<frame at 0x000001CD395F6440, file 'C:\\Users\\86182\\AppData\\Local\\Temp\\ipykernel_15624\\3920708156.py', line 29, code __init__>
ArgInfo(args=['self', 'xlabel', 'ylabel', 'xlim', 'ylim', 'xscale', 'yscale', 'ls', 'colors', 'fig', 'axes', 'figsize', 'display'], varargs=None, keywords=None, locals={'self': <__main__.ProgressBoard object at 0x000001CD3937BBF0>, 'xlabel': None, 'ylabel': None, 'xlim': None, 'ylim': None, 'xscale': 'linear', 'yscale': 'linear', 'ls': ['-', '--', '-.', ':'], 'colors': ['C0', 'C1', 'C2', 'C3'], 'fig': None, 'axes': None, 'figsize': (3.5, 2.5), 'display': True})
{'xlabel': None, 'ylabel': None, 'xlim': None, 'ylim': None, 'xscale': 'linear', 'yscale': 'linear', 'ls': ['-', '--', '-.', ':'], 'colors': ['C0', 'C1', 'C2', 'C3'], 'fig': None, 'axes': None, 'figsize': (3.5, 2.5), 'display': True}


In [6]:
"""
Must call requires_grad_ before calculation
"""
x = torch.tensor([1, 2.0])
y = x * x
x.requires_grad_(True)
print(y.grad_fn)

None


In [18]:
"""
Pytorch var and std are unbiased
"""
a = [1, 2.0]
print(np.var(a))
print((torch.tensor(a)).var())

0.25
tensor(0.5000)


In [6]:
"""
Adaptive pool
"""
m = nn.AdaptiveAvgPool2d((5, 7))
input_1 = torch.randn(1, 64, 8, 9)
input_2 = torch.randn(1, 8, 9)
output_1 = m(input_1)
output_2 = m(input_2)
print(output_1.shape, output_2.shape)

torch.Size([1, 64, 5, 7]) torch.Size([1, 5, 7])


In [9]:
"""
Flatten, start_dim = 1, end_dim = -1
"""
f = nn.Flatten()
print(f(torch.zeros(32, 10)).shape, f(torch.zeros(32, 10, 10)).shape, f(torch.zeros(32, 3, 28, 28)).shape)

torch.Size([32, 10]) torch.Size([32, 100]) torch.Size([32, 2352])


In [12]:
# Load DenseNet and ResNet models
densenet = models.densenet121(pretrained=False).cuda()
resnet = models.resnet50(pretrained=False).cuda()

# Create a dummy input of shape (batch_size, channels, height, width)
input_tensor = torch.randn(1, 3, 224, 224).cuda()

# Measure GPU memory usage
def measure_memory(model, input_tensor):
    torch.cuda.reset_max_memory_allocated()
    _ = model(input_tensor)
    return torch.cuda.max_memory_allocated()

print("DenseNet Memory Usage:", measure_memory(densenet, input_tensor))
print("ResNet Memory Usage:", measure_memory(resnet, input_tensor))

DenseNet Memory Usage: 278337536
ResNet Memory Usage: 258443264


In [ ]:
torch.cuda.reset_max_memory_allocated()

In [3]:
state = [1]
state, = state
state

1

In [27]:
one_hot = F.one_hot(torch.tensor([[0, 2, 3], [1, 2, 4]]).reshape(2, -1).T, 5)
print(one_hot)
print(torch.tensor([[0, 2, 3.], [1, 2, 4]]).reshape(-1,).shape)
print(one_hot.shape)
print(torch.reshape(one_hot, (-1, one_hot.shape[-1])).shape)
loss = F.cross_entropy(
            torch.reshape(one_hot, (-1, one_hot.shape[-1])).type(torch.float32), torch.tensor([[0, 2, 3], [1, 2, 4]]).reshape(-1,), reduction='mean')
print(loss)

tensor([[[1, 0, 0, 0, 0],
         [0, 1, 0, 0, 0]],

        [[0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0]],

        [[0, 0, 0, 1, 0],
         [0, 0, 0, 0, 1]]])
torch.Size([6])
torch.Size([3, 2, 5])
torch.Size([6, 5])
tensor(1.5715)


In [4]:
class TiedModel(nn.Module):
    def __init__(self):
        super().__init__()
        linear_layer = nn.LazyLinear(2)
        # self.net = nn.Sequential(linear_layer, nn.ReLU(),
        #                          linear_layer, nn.ReLU(),
        #                          nn.LazyLinear(1))
        self.net = nn.Sequential(nn.LazyLinear(2), nn.ReLU(),
                                 linear_layer, nn.ReLU(),
                                 linear_layer, nn.ReLU(),)
    def forward(self, x):
        return self.net(x)

x = torch.tensor([1., 2.])
x.requires_grad_(True)
model = TiedModel()
y = model(x)
y.sum().backward()
print(model.net[2].weight.grad)

tensor([[0.6370, 0.0000],
        [0.0000, 0.0000]])


In [13]:
z = torch.tensor([[1, 2.], [3, 4.]])
z.requires_grad_(True)
u = z @ z
v = u.sum()
u.retain_grad()
v.backward()
print(z.grad)
print(u.grad)
print(z @ u.grad)
print(u.grad @ z)

tensor([[ 7., 11.],
        [ 9., 13.]])
tensor([[1., 1.],
        [1., 1.]])
tensor([[3., 3.],
        [7., 7.]], grad_fn=<MmBackward0>)
tensor([[4., 6.],
        [4., 6.]], grad_fn=<MmBackward0>)


In [16]:
z1 = torch.tensor([[1, 2.], [3, 4.]])
z1.requires_grad_(True)
z2 = torch.tensor([[1, 2.], [3, 4.]])
z2.requires_grad_(True)
u = z1 @ z2
v = u.sum()
u.retain_grad()
v.backward()
print(z1.grad) # this is du/dv @ dv/dz1 -> ones(2, 2) @ z.T
print(z2.grad) # this is du/dv @ dv/dz2 -> z.T @ ones(2, 2)
print(u.grad)
print(u.grad @ z.T + z.T @ u.grad.T)
print(z1.grad + z2.grad)

tensor([[3., 7.],
        [3., 7.]])
tensor([[4., 4.],
        [6., 6.]])
tensor([[1., 1.],
        [1., 1.]])
tensor([[ 7., 11.],
        [ 9., 13.]], grad_fn=<AddBackward0>)
tensor([[ 7., 11.],
        [ 9., 13.]])


In [4]:
a = torch.tensor([[1, 2], [3, 4]])
print(a.sum(axis=0))

tensor([4, 6])
